## Installing the required dependencies

In [ ]:
!pip install -U pip setuptools wheel

!pip install -U spacy

!python -m spacy download en_core_web_sm

!pip install git+https://github.com/boudinfl/pke.git
!pip install matplotlib

     |████████████████████████████████| 2.1 MB 4.2 MB/s 
     |████████████████████████████████| 1.1 MB 33.2 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.0/452.0 KB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.3/653.3 KB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 KB 3.5 MB/s eta 0:00:00
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installa

# Importing the required libraries

In [1]:
import numpy as np
import pandas as pd
import pke
import spacy
spacy.load('en_core_web_sm')
import string
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **Initializing the dataset**

In [7]:
df1 = pd.read_csv("inspec_text.csv")
df2 = pd.read_csv("inspec_keys.csv", sep = "delimiter")

# Merge the two tables

df1["Key"] = df2
df = df1

df["Key"] = df["Key"].str.replace('"', "")
df["Text"] = df["Text"].str.replace("[", "")
df["Text"] = df["Text"].str.replace("]", "")
df["Text"] = df["Text"].str.replace("?", "")
df["Text"] = df["Text"].str.replace(",", "")
df["Text"] = df["Text"].str.lower()


#Display the dataset
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/li

,Text,Key
0,separate accounts go mainstream investmentnew ...,"classicism,universality,classical component of..."
1,does classicism explain universality arguments...,"philosophy of mind,information-based semantics..."
2,a conflict between language and atomistic info...,"world-making,selective representing,mental rep..."
3,selective representing and world-makingwe disc...,"lob theorem,limitation on mechanism,epistemic ..."
4,"lob's theorem as a limitation on mechanism""we ...","individual rationality,human behavior,game the..."


# Implementation of the algorithms:

# **Unsupervised**

## **Statistical Methods**

### **TF-IDF algorithm**

In [ ]:
# 1. create a TfIdf extractor.
extractor = pke.unsupervised.TfIdf()
keywords_tfidf = []
for index, row in df.iterrows():
  
  # 2. load the content of the document.
  extractor.load_document(input = row["Text"],
                          language='en',
                          normalization=None)

  # 3. select {1-3}-grams not containing punctuation marks as candidates.
  extractor.candidate_selection()

  # 4. weight the candidates using a `tf` x `idf`
  df = pke.load_document_frequency_file(input_file = "df-semeval2010.tsv.gz")
  extractor.candidate_weighting(df=df)

  # 5. get the 3-highest scored candidates as keyphrases
  keyphrases = extractor.get_n_best(n=3)
  keywords_tfidf.append(keyphrases)


In [ ]:
with open("key_inspec_tfidf.txt", "w") as external_file:
  for keyphrase in keywords_tfidf:   
    print(keyphrase, file=external_file)
  external_file.close()

### **KP-Miner algorithm**

In [ ]:
# 1. create a KPMiner extractor.
extractor = pke.unsupervised.KPMiner()
keywords_kpminer = []
for index, row in df.iterrows():
# 2. load the content of the document.
  extractor.load_document(input=row["Text"],
                        language='en',
                        normalization=None)


# 3. select {1-5}-grams that do not contain punctuation marks or
#    stopwords as keyphrase candidates. Set the least allowable seen
#    frequency to 5 and the number of words after which candidates are
#    filtered out to 200.

  extractor.candidate_selection(lasf=1, cutoff=400)

# 4. weight the candidates using KPMiner weighting function.
  df = pke.load_document_frequency_file(input_file='df-semeval2010.tsv.gz')
  alpha = 2.3
  sigma = 3.0
  extractor.candidate_weighting(df=df, alpha=alpha, sigma=sigma)

# 5. get the 3-highest scored candidates as keyphrases
  keyphrases = extractor.get_n_best(n=3)
  keywords_kpminer.append(keyphrases)


In [ ]:
with open("key_inspec_KPMiner.txt", "w") as external_file:
  for keyphrase in keywords_kpminer:   
    print(keyphrase, file=external_file)
  external_file.close()

## **Graph-Based Methods**

### **Text Rank algorithm**

In [ ]:
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a TextRank extractor.
extractor = pke.unsupervised.TextRank()

keywords_textrank = []
for index, row in df.iterrows():

  # 2. load the content of the document.
  extractor.load_document(input=row["Text"],
                          language='en',
                          normalization=None)

  # 3. build the graph representation of the document and rank the words.
  #    Keyphrase candidates are composed from the 33-percent
  #    highest-ranked words.
  extractor.candidate_weighting(window=2,
                                pos=pos,
                                top_percent=0.33)

  # 4. get the 3-highest scored candidates as keyphrases
  keyphrases = extractor.get_n_best(n=3)
  keywords_textrank.append(keyphrases)


In [ ]:
with open("key_inspec_textrank.txt", "w") as external_file:
  for keyphrase in keywords_textrank:   
    print(keyphrase, file=external_file)
  external_file.close()

### **Single Rank algorithm**

In [ ]:
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a SingleRank extractor.
extractor = pke.unsupervised.SingleRank()

keywords_singlerank = []
for index, row in df.iterrows():
  # 2. load the content of the document.
  extractor.load_document(input = row["Text"],
                          language='en',
                          normalization=None)

  # 3. select the longest sequences of nouns and adjectives as candidates.
  extractor.candidate_selection(pos=pos)

  # 4. weight the candidates using the sum of their word's scores that are
  #    computed using random walk. In the graph, nodes are words of
  #    certain part-of-speech (nouns and adjectives) that are connected if
  #    they occur in a window of 10 words.
  extractor.candidate_weighting(window=10,
                                pos=pos)

  # 5. get the 3-highest scored candidates as keyphrases
  keyphrases = extractor.get_n_best(n=3)
  keywords_singlerank.append(keyphrases)


In [ ]:
with open("key_inspec_singlerank.txt", "w") as external_file:
  for keyphrase in keywords_singlerank:   
    print(keyphrase, file=external_file)
  external_file.close()

### **Topic Rank algorithm**

In [ ]:
# 1. create a TopicRank extractor.
extractor = pke.unsupervised.TopicRank()

keywords_topicrank = []

for index, row in df.iterrows():
  # 2. load the content of the document.
  extractor.load_document(input = row["Text"])

  # 3. select the longest sequences of nouns and adjectives, that do
  #    not contain punctuation marks or stopwords as candidates.
  pos = {'NOUN', 'PROPN', 'ADJ'}
  stoplist = list(string.punctuation)
  stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
  stoplist += stopwords.words('english')
  extractor.candidate_selection(pos=pos)

  # 4. build topics by grouping candidates with HAC (average linkage,
  #    threshold of 1/4 of shared stems). Weight the topics using random
  #    walk, and select the first occuring candidate from each topic.
  extractor.candidate_weighting(threshold=0.74, method='average')

  # 5. get the 3-highest scored candidates as keyphrases
  keyphrases = extractor.get_n_best(n=3)
  keywords_topicrank.append(keyphrases)


In [ ]:
with open("key_inspec_topicrank.txt", "w") as external_file:
  for keyphrase in keywords_topicrank:   
    print(keyphrase, file=external_file)
  external_file.close()

### **Position Rank algorithm**

In [ ]:
# define the valid Part-of-Speeches to occur in the graph
pos = {'NOUN', 'PROPN', 'ADJ'}

# define the grammar for selecting the keyphrase candidates
grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"
extractor = pke.unsupervised.PositionRank()
keywords_positionrank = []
for index, row in df.iterrows():
# 1. create a PositionRank extractor.

# 2. load the content of the document.
  extractor.load_document(input=row["Text"],
                        language='en',
                        normalization=None)

# 3. select the noun phrases up to 3 words as keyphrase candidates.
  extractor.candidate_selection(grammar=grammar,
                              maximum_word_number=3)

# 4. weight the candidates using the sum of their word's scores that are
#    computed using random walk biaised with the position of the words
#    in the document. In the graph, nodes are words (nouns and
#    adjectives only) that are connected if they occur in a window of
#    10 words.
  extractor.candidate_weighting(window=10,
                              pos=pos)

# 5. get the 3-highest scored candidates as keyphrases
  keyphrases = extractor.get_n_best(n=3)
  keywords_positionrank.append(keyphrases)


In [ ]:
with open("key_inspec_positionrank.txt", "w") as external_file:
  for keyphrase in keywords_positionrank:   
    print(keyphrase, file=external_file)
  external_file.close()

# **Supervised**

## **Binary**

### **KEA algorithm**

In [ ]:
# define a list of stopwords
stoplist = stopwords.words('english')

# 1. create a Kea extractor.
extractor = pke.supervised.Kea()
keywords_kea = []
for index, row in df.iterrows():
# 2. load the content of the document.
  extractor.load_document(input=row["Text"],
                        language='en',
                        normalization=None)

# 3. select 1-3 grams that do not start or end with a stopword as
#    candidates. Candidates that contain punctuation marks as words
#    are discarded.
  extractor.candidate_selection()

# 4. classify candidates as keyphrase or not keyphrase.
  df = pke.load_document_frequency_file(input_file='df-semeval2010.tsv.gz')
  model_file = 'Kea-semeval2010.py3.pickle'
  extractor.candidate_weighting(model_file=model_file,
                              df=df)

# 5. get the 3-highest scored candidates as keyphrases
  keyphrases = extractor.get_n_best(n=3)
  keywords_kea.append(keyphrases)


In [ ]:
with open("key_inspec_kea.txt", "w") as external_file:
  for keyphrase in keywords_kea:   
    print(keyphrase, file=external_file)
  external_file.close()

# **YAKE! algorithm**

In [ ]:
# 1. create a YAKE extractor.
extractor = pke.unsupervised.YAKE()

keywords_yake = []
for index, row in df.iterrows():
# 2. load the content of the document.

  extractor.load_document(input=row["Text"],language='en')


# 3. select {1-3}-grams not containing punctuation marks and not
#    beginning/ending with a stopword as candidates.
  stoplist = stopwords.words('english')
  extractor.candidate_selection(n=3)

# 4. weight the candidates using YAKE weighting scheme, a window (in
#    words) for computing left/right contexts can be specified.
  window = 2
  use_stems = False # use stems instead of words for weighting
  extractor.candidate_weighting(window=2)

# 5. get the 3-highest scored candidates as keyphrases.
#    redundant keyphrases are removed from the output using levenshtein
#    distance and a threshold.
  threshold = 0.8
  keyphrases = extractor.get_n_best(n=3, threshold=threshold)
  keywords_yake.append(keyphrases)



In [ ]:
with open("key_inspec_yake.txt", "w") as external_file:
  for keyphrase in keywords_yake:   
    print(keyphrase, file=external_file)
  external_file.close()

# Data Preprocessing

In [ ]:
import re

In [ ]:
list=["inspec_kpminer.txt","inspec_positionrank.txt","inspec_yake.txt","inspec_kea.txt","inspec_tfidf.txt","inspec_topicrank.txt","inspec_textrank.txt","inspec_singlerank.txt"]
for file in list:
  with open(f'{file}', 'r') as infile,open(f'key_{file}', 'w') as outfile:
      data = infile.read()
      data = re.sub(r"[0-9]", "", data)
      data = data.replace("(", "").replace("),", "").replace(".","").replace("'","").replace("  "," ").replace("[","").replace("]","").replace(", )","")
      outfile.write(data)

In [ ]:
import pandas as pd
df1 = pd.read_csv("inspec_text.csv")
df2 = pd.read_csv("inspec_keys.csv", sep = "delimiter")
df3=pd.read_csv("key_inspec_tfidf.txt",sep = "delimiter", header=None)
df4=pd.read_csv("key_inspec_textrank.txt",sep = "delimiter", header=None)
df5=pd.read_csv("key_inspec_topicrank.txt",sep = "delimiter", header=None)
df6=pd.read_csv("key_inspec_singlerank.txt",sep = "delimiter", header=None)
df7=pd.read_csv("key_inspec_positionrank.txt",sep = "delimiter", header=None)
df8=pd.read_csv("key_inspec_kea.txt",sep = "delimiter", header=None)
df9=pd.read_csv("key_inspec_kpminer.txt",sep = "delimiter", header=None)
df10=pd.read_csv("key_inspec_yake.txt",sep = "delimiter", header=None)
# Merge all the tables

df1["Key"] = df2
df1["key_tfidf"]=df3
df1["key_textrank"]=df4
df1["key_topicrank"]=df5
df1["key_singlerank"]=df6
df1["key_positionrank"]=df7
df1["key_kea"]=df8
df1["key_KPMiner"]=df9
df1["key_yake"]=df10
df = df1

#Display the dataset
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,Text,Key,key_tfidf,key_textrank,key_topicrank,key_singlerank,key_positionrank,key_kea,key_KPMiner,key_yake
0,"Separate accounts go mainstream [investment],N...","classicism,universality,classical component of...","separate, separate accounts, accounts, mainstr...","investment,new entrants, separate accounts","tools, advisers, independent, web-based platfo...","separate-account industry, investment,new entr...","separate accounts, investment,new entrants, se...","separate, separate accounts, accounts, mainstr...","separate accounts, separate-account industry, ...","web-based platforms, money managers, separate ..."
1,Does classicism explain universality? Argument...,"philosophy of mind,Information-Based Semantics...","universality, classical, classicism, cognition...","classical symbol, syntax-sensitive rules, conn...","classicism, universality, human cognition, cap...","classical symbol systems, pure classical, univ...","pure classical, classical symbol systems, univ...","universality, classical, classicism, cognition...","universality, classical, classicism, cognition...","classicism explain universality, universality,..."
2,A conflict between language and atomistic info...,"world-making,selective representing,mental rep...","thesis, atomism, relations, ibs, representatio...","simple symbols, conflict ibs, content atomism,...","thesis, contents, content atomism, relations, ...","content atomism, cognitive states such, cognit...","conflict ibs, atomistic information, fred dret...","thesis, atomism, relations, ibs, representatio...","thesis, atomism, relations, ibs, representatio...","information-based semantics, fred dretske, jer..."
3,"Selective representing and world-making,We dis...","Lob Theorem,limitation on mechanism,epistemic ...","selective, representing, selective representin...","selective representing-the, realist conception...","selective representing, organisms, selective r...","selective representing-the idea, selective rep...","selective representing-the idea, selective rep...","selective, representing, selective representin...","selective, selective representing, mind-indepe...","selective representing-the idea, organisms evo..."
4,"Lob's theorem as a limitation on mechanism,""We...","individual rationality,human behavior,game the...","lob, observer, limitation, argue, mechanism, e...","identifiable formal, particular type, particul...","mechanism, theorem, lob, epistemic authority, ...","human-like epistemic agents, epistemic authori...","lob, theorem, limitation, mechanism,""we, parti...","lob, observer, limitation, argue, mechanism, e...","lob, observer, limitation, argue, mechanism, e...","theorem implies, theorem, lob, human-like epis..."


# Evaluation F1 Score 

In [ ]:
from sklearn.metrics import f1_score

df = pd.read_csv("inspec_text.csv")
df2 = pd.read_csv("inspec_keys.csv", sep = "delimiter")



df["Key"] = df2
df["key_tfidf"]=[f1_score(k, df3.values.tolist(), average='macro')  for k in df2.values.tolist()]
df["key_textrank"]=[f1_score(k, df4.values.tolist(), average='macro')  for k in df2.values.tolist()]
df["key_topicrank"]=[f1_score(k, df5.values.tolist(), average='macro')  for k in df2.values.tolist()]
df["key_singlerank"]=[f1_score(k, df6.values.tolist(), average='macro')  for k in df2.values.tolist()]
df["key_positionrank"]=[f1_score(k, df7.values.tolist(), average='macro')  for k in df2.values.tolist()]
df["key_kea"]=[f1_score(k, df8.values.tolist(), average='macro')  for k in df2.values.tolist()]
df["key_KPMiner"]=[f1_score(k, df9.values.tolist(), average='macro')  for k in df2.values.tolist()]
df["key_yake"]=[f1_score(k, df10.values.tolist(), average='macro')  for k in df2.values.tolist()]
df_fin = df

#Display the dataset
df_fin.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,Separate accounts go mainstream [investment],Key,key_tfidf,key_textrank,key_topicrank,key_singlerank,key_positionrank,key_kea,key_KPMiner,key_yake
0,New entrants are shaking up the separate-accou...,"classicism,universality,classical component of...",0.167516,0.384862,0.427431,0.060107,0.405758,0.415933,0.210590,0.107248
1,\tWeb-based platforms that give advisers the t...,"philosophy of mind,Information-Based Semantics...",0.000154,0.433208,0.369200,0.165796,0.308411,0.424268,0.192254,0.348618
2,\tmoney managers,"world-making,selective representing,mental rep...",0.058150,0.352413,0.270179,0.345800,0.338215,0.192560,0.237568,0.294434
3,Does classicism explain universality? Argument...,"Lob Theorem,limitation on mechanism,epistemic ...",0.314396,0.233082,0.177040,0.014236,0.189688,0.031453,0.164483,0.397296
4,\tcomponent of mind,"individual rationality,human behavior,game the...",0.447226,0.446228,0.199438,0.422453,0.107050,0.150939,0.121319,0.431726


In [ ]:
import pandas as pd
d = {'key_tfidf': [df["key_tfidf"].mean()], 'key_textrank': [df["key_textrank"].mean()], 'key_topicrank': [df["key_topicrank"].mean()], 'key_singlerank': [df["key_singlerank"].mean()], 'key_positionrank': [df["key_positionrank"].mean()], 'key_kea': [df["key_kea"].mean()], 'key_KPMiner': [df["key_KPMiner"].mean()], 'key_yake': [df["key_yake"].mean()]}
df_score = pd.DataFrame(data=d)
df_score.head()

,key_tfidf,key_textrank,key_topicrank,key_singlerank,key_positionrank,key_kea,key_KPMiner,key_yake
0,0.15432,0.102366,0.242197,0.359631,0.310233,0.135477,0.098722,0.39321
